In [1]:
import simtk.openmm.app as app
import simtk.openmm as mm
import simtk.unit as unit
import mdtraj as md
import matplotlib.pyplot as plt
import nglview as nv

_ColormakerRegistry()

## Cargar el archivo PDB original

El PDB original suponemos que fue minimizado con forcefield Charmm XX viejo

In [2]:
pdb =app.PDBFile("BAK_model1_all.pdb")

## Creamos los pdbs de los sistemas a comparar

In [3]:
experiments={
    'Amber14_vacuum': [["amber14-all.xml"], 'vacuum'],
    'Amber14_implicit': [["amber14-all.xml"], 'implicit'],
    'Amber14_explicit': [["amber14-all.xml", "amber14/tip3p.xml"], 'explicit'],
    'Charmm36_vacuum': [["charmm36.xml"], 'vacuum'],
    'Charmm36_implicit': [["charmm36.xml"], 'implicit'],
    'Charmm36_explicit': [["charmm36.xml","charmm36/water.xml"], 'explicit'],
}

In [4]:
vacuum_parameters={
    'nonbondedMethod': app.NoCutoff,
    'constraints': app.HBonds
}

implicit_parameters={
    'nonbondedMethod': app.NoCutoff,
    'constraints': app.HBonds,
    'implicitSolvent': app.OBC2
}

explicit_parameters={
    'nonbondedMethod': app.PME,
    'nonbondedCutoff': 1*unit.nanometer,
    'constraints': app.HBonds
}

In [5]:
verbose = True

In [6]:
for experiment, conditions in experiments.items():
    
    if verbose:
        print("Doing experiment: {}".format(experiment))
    
    list_forcefield_files = conditions[0]
    solvent = conditions[1]
    
    topology = pdb.topology
    positions = pdb.positions
    
    # Forcefield
    
    forcefield =app.ForceField(*list_forcefield_files)
    
    # Adding solvent if experiment is explicit
    
    if solvent=='explicit':
        
        modeller = app.Modeller(pdb.topology, pdb.positions)
        modeller.addSolvent(forcefield, padding=1.4*unit.nanometers)
        
        topology = modeller.topology
        positions = modeller.positions
    
    # Creating system
    
    creation_parameters = None
    
    if solvent=='vacuum':
        creation_parameters = vacuum_parameters
    elif solvent=='implicit':
        creation_parameters = implicit_parameters
    elif solvent=='explicit':
        creation_parameters = explicit_parameters
    
    system = forcefield.createSystem(topology, **creation_parameters)
    
    # Integrator
    
    integrator = mm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picoseconds, 2.0*unit.femtoseconds)
    integrator.setConstraintTolerance(0.00001)
    
    # Platform
    
    platform = mm.Platform.getPlatformByName('CPU')
    #properties = {'CudaPrecision': 'mixed'}
    properties = {}
    
    # Simulation
    
    simulation = app.Simulation(topology, system, integrator, platform, properties)
    simulation.context.setPositions(positions)
    
    # Minimization
    
    if verbose:
        Edo_PreMin = simulation.context.getState(getEnergy = True)
        E = Edo_PreMin.getPotentialEnergy()
        print(E)
    
    if verbose:
        print('Minimizing...')
    
    simulation.minimizeEnergy(tolerance=0.1*unit.kilojoule/unit.mole, maxIterations=500)
    
    Edo_PosMin = simulation.context.getState(getEnergy = True, getPositions = True)
    
    if verbose:
        E = Edo_PosMin.getPotentialEnergy()
        print(E)
    
    positions_min = Edo_PosMin.getPositions()
    
    # Saving the minimized system in new pdb file
    
    pdbfile_path = 'PDBs_experiments/'+experiment+'.pdb'
    app.PDBFile.writeFile(topology, positions_min, open(pdbfile_path, 'w'))
    
    if verbose:
        print('... Done')
        print(' ')

Doing experiment: Amber14_vacuum
9145.484760688047 kJ/mol
Minimizing...
-6131.222027824632 kJ/mol
... Done
 
Doing experiment: Amber14_implicit
9145.484760688047 kJ/mol
Minimizing...
-6175.91228761893 kJ/mol
... Done
 
Doing experiment: Amber14_explicit
-182642.73067152264 kJ/mol
Minimizing...
-458840.58221635816 kJ/mol
... Done
 
Doing experiment: Charmm36_vacuum
7386.410383416282 kJ/mol
Minimizing...
-7316.532659292166 kJ/mol
... Done
 
Doing experiment: Charmm36_implicit
7386.410383416283 kJ/mol
Minimizing...
-7270.913488211908 kJ/mol
... Done
 
Doing experiment: Charmm36_explicit
-141799.59449282679 kJ/mol
Minimizing...
-479149.4558802692 kJ/mol
... Done
 
